# Importing The Libraries

In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Activation, Dropout, Flatten, Dense, BatchNormalization

In [2]:
import mediapipe as mp
import uuid

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Things that will make sense later

In [4]:
X=[]
Y=[]

In [5]:
def oldListToNewList(oldList):
    newList=[]
    oldList = np.array(oldList)
    for i in range(20):
        for j in range(i+1,21):
            newList.append(((oldList[i][0]-oldList[j][0])**2+(oldList[i][1]-oldList[j][1])**2))
    for i in range(21):
        newList.append(oldList[i][2])
    if (len(oldList)==0):
        print("Zero")
    return newList

I've recorded videos of my hand to make the training data, This piece will just read those videos and get the hand co-ordinates of every letters, then getting the distance between the 21 points that we get and their z co-ordinates and storint it in an array for our training data.

Here's a reference video link which uses the media pipe to get co-ordinates of hands :- https://youtu.be/p5Z_GGRCI5s

In [6]:
alphabets="abcdefghijklmnopqrstuvwxyz"
for i in range(len(alphabets)):
    mypath='D:/HandPhotos/'+alphabets[i]
    onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
    for n in range(0, len(onlyfiles)):
        img=cv2.imread(join(mypath,onlyfiles[n]))
        img = cv2.resize(img, (1000, 600))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
            results = hands.process(img)
        ListOfCoOrdinates=[]
        if (results.multi_hand_landmarks is not None): 
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_CMC].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_CMC].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_CMC].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_MCP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_MCP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_MCP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_IP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_IP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_IP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_MCP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_MCP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_PIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_PIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_DIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_DIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_TIP].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP ].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_PIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_PIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_PIP ].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_DIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_DIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_DIP ].z])
            ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_TIP].x,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_TIP].y,
                results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_TIP ].z])
            X.append(oldListToNewList(ListOfCoOrdinates))
            Y.append(i)
    print(alphabets[i])

a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z


# Basic Structuring of Training Data

In [7]:
X=np.array(X)
Y=np.array(Y)

In [8]:
X, Y = shuffle(X, Y)

In [9]:
Y.shape

(45543,)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [11]:
x_train = x_train.reshape(-1, 231)
x_test = x_test.reshape(-1, 231)

# Simple Deep Learning Model

In [12]:
model = Sequential([
    Dense(26, input_shape=(210+21,), activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=40)

Epoch 1/40
1139/1139 [==============================] - 1s 518us/step - loss: 2.0682 - accuracy: 0.6970
Epoch 2/40
1139/1139 [==============================] - 1s 512us/step - loss: 1.0456 - accuracy: 0.9276
Epoch 3/40
1139/1139 [==============================] - 1s 509us/step - loss: 0.6785 - accuracy: 0.9590
Epoch 4/40
1139/1139 [==============================] - 1s 514us/step - loss: 0.4901 - accuracy: 0.9697
Epoch 5/40
1139/1139 [==============================] - 1s 513us/step - loss: 0.3760 - accuracy: 0.9740
Epoch 6/40
1139/1139 [==============================] - 1s 530us/step - loss: 0.3005 - accuracy: 0.9768
Epoch 7/40
1139/1139 [==============================] - 1s 529us/step - loss: 0.2480 - accuracy: 0.9796
Epoch 8/40
1139/1139 [==============================] - 1s 509us/step - loss: 0.2096 - accuracy: 0.9806
Epoch 9/40
1139/1139 [==============================] - 1s 599us/step - loss: 0.1808 - accuracy: 0.9822
Epoch 10/40
1139/1139 [==============================] - 1s 555u

In [13]:
model.evaluate(x_test, y_test)

285/285 [==============================] - 0s 527us/step - loss: 0.0412 - accuracy: 0.9936


[0.0411786250770092, 0.9936326742172241]

# The Final Output Video 

In [14]:
imageLinks=[]
for i in range(26):
    mypath='D:/HandPhotos/outputImages/'+alphabets[i]+'.jpg'
    img=cv2.imread(mypath)
    imageLinks.append(img)
nothing=cv2.imread('D:/HandPhotos/outputImages/nothing.jpg')

In [ ]:
# your video link
cam = cv2.VideoCapture(r'D:\HandPhotos\output2.mp4') 
while(True): 

    # reading from frame 
    ret,frame = cam.read() 
    
    if ret: 
        # if video is still left continue creating images 
        frame = cv2.resize(frame, (1000, 600))
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        with mp_hands.Hands(min_detection_confidence=0.9, min_tracking_confidence=0.8) as hands:
            results = hands.process(img)
            ListOfCoOrdinates=[]
            if (results.multi_hand_landmarks!=None):
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_CMC].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_CMC].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_CMC].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_MCP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_MCP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_MCP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_IP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_IP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_IP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_MCP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_MCP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_PIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_PIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_DIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_DIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_TIP].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP ].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_PIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_PIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_PIP ].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_DIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_DIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_DIP ].z])
                ListOfCoOrdinates.append([results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_TIP].x,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_TIP].y,
                    results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_TIP ].z])
                stored=oldListToNewList(ListOfCoOrdinates)
                stored=np.array(stored)
                stored = stored.reshape(-1, 231)
                image = cv2.resize(frame, (1000, 600))
                if (max(model.predict(stored)[0])>=0.6):
                    predicted=np.argmax(model.predict(stored)[0])
                    image[0:200,0:200]=imageLinks[predicted]
                else:
                    image[0:200,0:200]=nothing
            else:
                image = cv2.resize(frame, (1000, 600))
                image[0:200,0:200]=nothing
            # image=cv2.putText(image,text,(100,100),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
            cv2.imshow('Hand Tracking', image)
            cv2.waitKey(1)
    else: 
        break

# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows() 